# Topics for March 2, 2016

My goal is to provide fast methods to use Markov-Chain Monte Carlo (MCMC) methods on linear mixed-effects models, with possible extensions to generalized linear mixed models.

There are several MCMC frameworks for [Julia](http://julialang.org).  An interface to [Stan](http://mc-stan.org) is available and there are two native Julia implementations; [Mamba](https://github.com/brian-j-smith/Mamba.jl) and [Lora](https://github.com/JuliaStats/Lora.jl).  I prefer `Mamba` because of its flexibility.  The problem with Stan, BUGS and JAGS is that each of them reinvents all the data structures, input/output, data manipulation, distribution definitions, etc. in its own environment.  They also define a Domain Specific Language (DSL) for which they must provide parsers, interpreters, run-time environments, etc.

A native implementation like Mamba can use all of the facilities of Julia and its packages.

## Linear predictor

Whenever you have a linear predictor (i.e. an $\bf X\beta$ type of expression) in a model there is a good chance that you can write out the full conditional distribution of $\beta$ or obtain a good approximation to it.  If you can write out the conditional distribution you can use a multivariate Gibbs sampler to obtain a vector-valued sample from the condtional.  This helps to avoid one of the underlying problems of MCMC methods which is successive sampling from conditionals of correlated parameters.  Consider a case where you might have hundreds or thousands of random effects and dozens of fixed effects.  You don't want to sample sequentially in those cases when you can sample from the distribution of the entire vector in one step.

## Multivariate normal conditionals

In most cases the conditional distribution of the coefficients (i.e. both random and fixed effects) is a multivariate normal with known mean and covariance matrix.  It is worthwhile examining the representation in Julia of this distribution.  Not surprisingly the representation involved the mean and covariance but the form of the covariance is encoded in the type.  For example, a common prior distribution for coefficients is a zero-mean multivariate normal with a covariance that is a large multiple of the identity - a diffuse prior.

In [1]:
versioninfo(true)

 [inlined code] from ./error.jl:26
 in depwarn(::ASCIIString, ::Symbol) at ./deprecated.jl:64
 in readbytes(::Base.PipeEndpoint, ::Vararg{Any}) at ./deprecated.jl:30
 in send_stream(::Base.PipeEndpoint, ::ASCIIString) at /home/bates/.julia/v0.5/IJulia/src/stdio.jl:25
 in 

Julia Version 0.5.0-dev+3149


watch_stream(::Base.PipeEndpoint, ::ASCIIString) at /home/bates/.julia/v0.5/IJulia/src/stdio.jl:41
 in (::IJulia.##4#8)() at ./task.jl:431
while loading In[1], in expression starting on line 1


Commit 3c72273* (2016-03-14 16:15 UTC)
Platform Info:
  System: Linux (x86_64-linux-gnu)
  CPU: Intel(R) Core(TM) i5-3570 CPU @ 3.40GHz
  WORD_SIZE: 64
           Ubuntu 15.10
  uname: Linux 4.2.0-34-generic #39-Ubuntu SMP Thu Mar 10 22:13:01 UTC 2016 x86_64 x86_64
Memory: 15.561424255371094 GB (10432.33203125 MB free)
Uptime: 3851.0 sec
Load Avg:  0.65771484375  0.4658203125  0.5283203125
Intel(R) Core(TM) i5-3570 CPU @ 3.40GHz: 
       speed         user         nice          sys         idle          irq
#1  3798 MHz      21737 s        540 s       3672 s     341064 s          0 s
#2  3698 MHz      33490 s        990 s       3509 s     329140 s          0 s
#3  3689 MHz      19396 s        465 s       3304 s     350182 s          0 s
#4  3692 MHz      20761 s        454 s       3654 s     348260 s          0 s

  BLAS: libopenblas (NO_LAPACKE DYNAMIC_ARCH NO_AFFINITY Sandybridge)
  LAPACK: liblapack
  LIBM: libopenlibm
  LLVM: libLLVM-3.7.1 (ORCJIT, ivybridge)
Environment:
  XDG_SES

In [2]:
using Distributions, GLM, Mamba, PDMats, StatsBase
d = MvNormal(2, 1000.)

    write(GZip.GZipStream, Array{#T<:Any, N<:Any}) at /home/bates/.julia/v0.5/GZip/src/GZip.jl:456
is ambiguous with: 
    write(Base.IO, Array{UInt8, N<:Any}) at io.jl:154.
To fix, define 
    write(GZip.GZipStream, Array{UInt8, N<:Any})
before the new definition.


ZeroMeanIsoNormal(
dim: 2
μ: [0.0,0.0]
Σ: 2x2 Array{Float64,2}:
 1.0e6  0.0  
 0.0    1.0e6
)


If you take apart the representation itself you discover that the only values that are stored are the scalar $\sigma^2$ and its inverse.

In [3]:
fieldnames(d)

2-element Array{Symbol,1}:
 :μ
 :Σ

In [4]:
typeof(d.μ)

Distributions.ZeroVector{Float64}

In [5]:
typeof(d.Σ)

PDMats.ScalMat{Float64}

In [6]:
fieldnames(d.Σ)

3-element Array{Symbol,1}:
 :dim      
 :value    
 :inv_value

In [7]:
(d.Σ.dim, d.Σ.value, d.Σ.inv_value)

(2,1.0e6,1.0e-6)

## Sampling without a prior

Let's defer the issue of a prior for a moment and consider that we have the matrix $\bf X'X$, the vector $\bf X'y$ and the scalar $\sigma$ defining the log-likelihood.  Without the prior, the mean is

In [8]:
X = hcat(ones(5), [1.:5;])

5x2 Array{Float64,2}:
 1.0  1.0
 1.0  2.0
 1.0  3.0
 1.0  4.0
 1.0  5.0

In [9]:
XtX = PDMat(X'X)

PDMats.PDMat{Float64,Array{Float64,2}}(2,2x2 Array{Float64,2}:
  5.0  15.0
 15.0  55.0,Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
2x2 UpperTriangular{Float64,Array{Float64,2}}:
 2.23607  6.7082 
  ⋅       3.16228)

In [10]:
y = [1.,3,3,3,5];
Xty = X'y

2-element Array{Float64,1}:
 15.0
 53.0

In [11]:
β = XtX\Xty

2-element Array{Float64,1}:
 0.6
 0.8

We could obtain the residual sum of squares by forming $\mu = \bf X'\beta$ but there is a short cut that we can use later with mixed-effects models.  We append the column of $\bf y$ to the $\bf X$ and then form the Cholesky decomposition.

In [12]:
const Xy = hcat(X,y)

5x3 Array{Float64,2}:
 1.0  1.0  1.0
 1.0  2.0  3.0
 1.0  3.0  3.0
 1.0  4.0  3.0
 1.0  5.0  5.0

In [13]:
pp = PDMat(Xy'Xy)

PDMats.PDMat{Float64,Array{Float64,2}}(3,3x3 Array{Float64,2}:
  5.0  15.0  15.0
 15.0  55.0  53.0
 15.0  53.0  53.0,Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
3x3 UpperTriangular{Float64,Array{Float64,2}}:
 2.23607  6.7082   6.7082 
  ⋅       3.16228  2.52982
  ⋅        ⋅       1.26491)

This `PDMat` object contains the positive-definite matrix itself and its (upper) Cholesky factor.

In [14]:
fieldnames(pp)

3-element Array{Symbol,1}:
 :dim 
 :mat 
 :chol

In [15]:
pp.mat

3x3 Array{Float64,2}:
  5.0  15.0  15.0
 15.0  55.0  53.0
 15.0  53.0  53.0

In [16]:
pp.chol

Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
3x3 UpperTriangular{Float64,Array{Float64,2}}:
 2.23607  6.7082   6.7082 
  ⋅       3.16228  2.52982
  ⋅        ⋅       1.26491

As with many of the [factorizations](http://docs.julialang.org/en/latest/manual/linear-algebra/) available in Julia ([see also](http://docs.julialang.org/en/latest/stdlib/linalg/#stdlib-linalg)), the `Cholesky` factorization provides the factor itself by indexing with a symbol.

In [17]:
R = pp.chol[:U]

3x3 UpperTriangular{Float64,Array{Float64,2}}:
 2.23607  6.7082   6.7082 
  ⋅       3.16228  2.52982
  ⋅        ⋅       1.26491

The [3, 3] element is the square root of the residual sum of squares.

In [18]:
abs2(R[3, 3]) # abs2 is x^2 as a function

1.6000000000000012

In [19]:
using StatsBase
sqL2dist(y, X * β)  # squared L₂ distance

1.5999999999999992

In [20]:
sqL2dist(y, X * β) ≈ abs2(R[3, 3])

true

The least squares estimates of the coefficients are obtained as

In [21]:
R12 = UpperTriangular(R[1:2, 1:2]);
typeof(R12)

UpperTriangular{Float64,Array{Float64,2}}

In [22]:
bb = R12 \ R[1:2, 3]

2-element Array{Float64,1}:
 0.6
 0.8

Furthermore, the covariance matrix of the least squares parameter estimator is 

In [23]:
R12inv = inv(R12)

2x2 UpperTriangular{Float64,Array{Float64,2}}:
 0.447214  -0.948683
  ⋅         0.316228

In [24]:
Σ₁ = unscaledΣ = PDMat(R12inv * R12inv')

PDMats.PDMat{Float64,Array{Float64,2}}(2,2x2 Array{Float64,2}:
  1.1  -0.3
 -0.3   0.1,Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
2x2 UpperTriangular{Float64,Array{Float64,2}}:
 1.04881  -0.286039
  ⋅        0.13484 )

In [25]:
inv(PDMat(X'X))

PDMats.PDMat{Float64,Array{Float64,2}}(2,2x2 Array{Float64,2}:
  1.1  -0.3
 -0.3   0.1,Base.LinAlg.Cholesky{Float64,Array{Float64,2}} with factor:
2x2 UpperTriangular{Float64,Array{Float64,2}}:
 1.04881  -0.286039
  ⋅        0.13484 )

We want to sample from a $\mathcal{N}(\beta, \sigma^2 \Sigma_1)$ distribution for the Gibbs update of $\beta$.

## Sampling from a multivariate normal distribution

At this point we want to sample from
If you trace back through methods for functions like `rand`, its mutating version `rand!` and its hidden, mutating version without dimension checks, `_rand!`, you will find that a call to
```julia
rand(MvNormal(β, abs2(σ) * Σ₁))
```
eventually calls
```julia
_rand!(d::MvNormal, x::VecOrMat{Float64}) = add!(unwhiten!(d.Σ, randn!(x)), d.μ)
```
where `randn!(x)` overwrites `x` with $\mathcal{N}(0, I)$ random variates.

This brings us to the question of what is "unwhitening"?  Well, it is the inverse of "whitening" which is producing "white noise", i.e. uncorrelated, constant variance multivariate normal disturbances, from correlated, non-constant variance disturbances.  

This is the multivariate equivalent of the inverse of the univariate "z transformation"
$$
x = \mu + \sigma z
$$

For the multivariate case we need to multiply a vector $\mathbf{z}$ by $\sigma$ and a "square root" of $\Sigma_1$, which is given by its Cholesky factor, $R_1$.  That is $\mathrm{Var}(R_1\mathbf{z})$ is $R_1'\mathrm{Var}(\mathbf{zz}')R_1=R_1'R_1=\Sigma_1$.

In [30]:
σ = R[3, 3]/√3

0.7302967433402218